## Aprendizaje de reglas de asociación

Objetivo:
    
    derivar reglas de la forma {A} -> {B}
    

In [1]:
import numpy as np
import os
import pandas as pd
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display
from sklearn.model_selection import train_test_split
from efficient_apriori import apriori

In [2]:
CURRENT_DIR = os.path.dirname('__file__')
DATASET_PATH = os.path.join(CURRENT_DIR, 'ml-20m')
movies = pd.read_csv(os.path.join(DATASET_PATH, 'movies.csv'))
ratings = pd.read_csv(os.path.join(DATASET_PATH, 'ratings.csv'))

Primero que nada, vamos a realizar un breve análisis de los datasets a utilizar

In [3]:
movies.describe()

,movieId
count,27278.000000
mean,59855.480570
std,44429.314697
min,1.000000
25%,6931.250000
50%,68068.000000
75%,100293.250000
max,131262.000000


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.describe()

,userId,movieId,rating,timestamp
count,2.000026e+07,2.000026e+07,2.000026e+07,2.000026e+07
mean,6.904587e+04,9.041567e+03,3.525529e+00,1.100918e+09
std,4.003863e+04,1.978948e+04,1.051989e+00,1.621694e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,3.439500e+04,9.020000e+02,3.000000e+00,9.667977e+08
50%,6.914100e+04,2.167000e+03,3.500000e+00,1.103556e+09
75%,1.036370e+05,4.770000e+03,4.000000e+00,1.225642e+09
max,1.384930e+05,1.312620e+05,5.000000e+00,1.427784e+09


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


Vamos a partir el dataset y analizar solamente el 10% de las películas del total de películas de movielens

In [7]:
# Seteamos valor de la semilla aleatoria para hacer repetible el experimento
np.random.seed(1234)
train, movies_10 = train_test_split(movies, test_size=0.1)

Vamos a emparejar las calificaciones de los usuarios con las películas

In [8]:
movies_df = pd.merge(ratings[['userId','movieId']], movies_10[['movieId','title']] ,on='movieId', how='inner')

display(movies_df.head())

movies_df = movies_df.sort_values( by='userId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
movies_df.drop('movieId', axis=1, inplace=True)
movies_df = movies_df.values[:,[0,1]]
print(movies_df)

,userId,movieId,title
0,1,260,Star Wars: Episode IV - A New Hope (1977)
1,2,260,Star Wars: Episode IV - A New Hope (1977)
2,3,260,Star Wars: Episode IV - A New Hope (1977)
3,5,260,Star Wars: Episode IV - A New Hope (1977)
4,6,260,Star Wars: Episode IV - A New Hope (1977)


[[1 'Star Wars: Episode IV - A New Hope (1977)']
 [1 'Evil Dead II (Dead by Dawn) (1987)']
 [1 'Indiana Jones and the Last Crusade (1989)']
 ...
 [138493 'Trainspotting (1996)']
 [138493 'Jesus Christ Superstar (1973)']
 [138493 'Ice Age: Dawn of the Dinosaurs (2009)']]


Vamos a generar el conjunto de transacciones

In [10]:
transactions = {}

for user_id, movie in movies_df:
    if user_id not in transactions:
        transactions[user_id] = []
    transactions[user_id].append(movie)

transactions = [movies for user_id, movies in transactions.items()]

In [11]:
transactions

[['Star Wars: Episode IV - A New Hope (1977)',
  'Evil Dead II (Dead by Dawn) (1987)',
  'Indiana Jones and the Last Crusade (1989)',
  'Jaws (1975)',
  'Star Kid (1997)',
  'Borrowers, The (1997)',
  'Iron Giant, The (1999)',
  'Sleepy Hollow (1999)',
  'O Brother, Where Art Thou? (2000)',
  'Memento (2000)',
  'Wicker Man, The (1973)',
  '28 Days Later (2002)',
  'Peter Pan (2003)',
  'Van Helsing (2004)',
  'Slaughterhouse-Five (1972)',
  'Rosencrantz and Guildenstern Are Dead (1990)',
  'Ran (1985)'],
 ['Return of the Fly (1959)',
  'Star Wars: Episode IV - A New Hope (1977)',
  "Nightmare on Elm Street 2: Freddy's Revenge, A (1985)",
  'Fantasia 2000 (1999)'],
 ['Radio Days (1987)',
  'Instinct (1999)',
  'Cruel Intentions (1999)',
  'Trading Places (1983)',
  'Deep Rising (1998)',
  'Sleeper (1973)',
  'Fugitive, The (1993)',
  'Kids (1995)',
  'Primary Colors (1998)',
  'Star Wars: Episode IV - A New Hope (1977)',
  'Evil Dead II (Dead by Dawn) (1987)',
  'Universal Soldier (199

Calculamos con el algoritmo a priori

In [20]:
#wARNING-CUIDADO!! no poner min_support pequeño como por ejempo 0.001!!
#min_support y min_confiden entre 0 y 1  
itemsets, rules = apriori(transactions, min_support=0.05,  min_confidence=0.6, max_length=3)

In [21]:
#rules = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
rules = sorted(rules, key=lambda rule: rule.confidence, reverse=True)
for rule in rules:
  print(rule) # Prints the rule and its confidence, support, lift, ...

{Client, The (1994), Natural Born Killers (1994)} -> {Fugitive, The (1993)} (conf: 0.931, supp: 0.052, lift: 2.535, conv: 9.198)
{Firm, The (1993), Natural Born Killers (1994)} -> {Fugitive, The (1993)} (conf: 0.914, supp: 0.084, lift: 2.489, conv: 7.366)
{Client, The (1994), Firm, The (1993)} -> {Fugitive, The (1993)} (conf: 0.913, supp: 0.081, lift: 2.485, conv: 7.261)
{Indiana Jones and the Last Crusade (1989), Jaws (1975)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.904, supp: 0.098, lift: 2.239, conv: 6.218)
{Client, The (1994), Mrs. Doubtfire (1993)} -> {Fugitive, The (1993)} (conf: 0.902, supp: 0.073, lift: 2.457, conv: 6.483)
{Client, The (1994), Pretty Woman (1990)} -> {Fugitive, The (1993)} (conf: 0.898, supp: 0.072, lift: 2.445, conv: 6.212)
{Indiana Jones and the Last Crusade (1989), Natural Born Killers (1994)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.898, supp: 0.057, lift: 2.224, conv: 5.852)
{Air Force One (1997), Indiana Jones and the Last Cr

In [22]:
#rules = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
rules = sorted(rules, key=lambda rule: rule.support, reverse=True)
for rule in rules:
  print(rule) # Prints the rule and its confidence, support, lift, ...

{Indiana Jones and the Last Crusade (1989)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.813, supp: 0.188, lift: 2.013, conv: 3.185)
{Mrs. Doubtfire (1993)} -> {Fugitive, The (1993)} (conf: 0.689, supp: 0.176, lift: 1.877, conv: 2.037)
{Pretty Woman (1990)} -> {Mrs. Doubtfire (1993)} (conf: 0.690, supp: 0.173, lift: 2.707, conv: 2.401)
{Mrs. Doubtfire (1993)} -> {Pretty Woman (1990)} (conf: 0.680, supp: 0.173, lift: 2.707, conv: 2.339)
{Pretty Woman (1990)} -> {Fugitive, The (1993)} (conf: 0.684, supp: 0.172, lift: 1.861, conv: 2.000)
{Firm, The (1993)} -> {Fugitive, The (1993)} (conf: 0.847, supp: 0.161, lift: 2.305, conv: 4.129)
{Fifth Element, The (1997)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.738, supp: 0.151, lift: 1.828, conv: 2.276)
{Indiana Jones and the Last Crusade (1989)} -> {Fugitive, The (1993)} (conf: 0.606, supp: 0.140, lift: 1.650, conv: 1.606)
{Memento (2000)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.603, supp: 0.136, lift: 1.

In [24]:
#rules = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
rules = sorted(rules, key=lambda rule: rule.lift, reverse=True)
for rule in rules:
  print(rule) # Prints the rule and its confidence, support, lift, ...

{Almost Famous (2000), Memento (2000)} -> {O Brother, Where Art Thou? (2000)} (conf: 0.655, supp: 0.054, lift: 4.543, conv: 2.480)
{Client, The (1994), Pretty Woman (1990)} -> {Firm, The (1993)} (conf: 0.850, supp: 0.068, lift: 4.465, conv: 5.391)
{Client, The (1994), Mrs. Doubtfire (1993)} -> {Firm, The (1993)} (conf: 0.840, supp: 0.068, lift: 4.414, conv: 5.060)
{Client, The (1994), Fugitive, The (1993)} -> {Firm, The (1993)} (conf: 0.825, supp: 0.081, lift: 4.337, conv: 4.636)
{Almost Famous (2000), Star Wars: Episode IV - A New Hope (1977)} -> {O Brother, Where Art Thou? (2000)} (conf: 0.624, supp: 0.052, lift: 4.331, conv: 2.278)
{Client, The (1994)} -> {Firm, The (1993), Fugitive, The (1993)} (conf: 0.690, supp: 0.081, lift: 4.279, conv: 2.702)
{Client, The (1994)} -> {Firm, The (1993)} (conf: 0.755, supp: 0.089, lift: 3.969, conv: 3.309)
{Memento (2000), Mrs. Doubtfire (1993)} -> {Monsters, Inc. (2001)} (conf: 0.656, supp: 0.052, lift: 3.742, conv: 2.396)
{Indiana Jones and the 

El trabajo práctico constaba de analizar las películas en el dataset de movielens y las calificaciones de los usuarios, para poder elaborar un sistema de recomendación.
En principio, el dataset de movielens era muy grande para poder realizar el análisis, por lo que se decidió tomar únicamente el 10% del total de las películas. Para esto, es necesario organizar la información en transacciones, que nos indican que películas calificó cada usuario.
En base al conjunto de películas que calificó cada uno de los usuarios, se procedió a armar el conjuto de reglas utilizando el algoritmo 'apriori'.
Para ejecutar el algoritmo, decidimos poner la confianza mínima en 0.6, ya que había una gran cantidad de reglas con una confianza mayor a este valor, y valores más pequeños podrían ser poco relevantes.
El soporte mínimo fue tomado con un valor de 0.05, ya que comenzamos con un valor más alto y lo fuimos disminuyendo para aumentar la cantidad de reglas obtenidas. No utilizamos un valor muy bajo ya que al usar valores más altos, el tiempo de procesamiento era muy alto. Por esta misma razón, también decidimos utilizar un max_length de 3.
Luego analizamos un poco las reglas obtenidas, ordenando de mayor a menor por algunas de las metricas que contiene cada regla.
Para poder observar cuales eran los valores más representativos, optamos por ordenar por confianza primero, ya que esto nos indica que a partir de que un usuario calificó la película X, calificó además la película Y. Un valor alto de confianza nos indicaría que la gran cantidad de usuarios calificaron ambas películas, con lo cual sería bueno recomendarle a alguien la película Y, dado que vio la película X.
Luego ordenamos también por soporte. El soporte nos indica que proporción de usuarios calificaron todas las películas de la regla. Un valor alto en el soporte entonces nos indicaría que una proporción significativa de los usuarios calificaron ese mismo conjunto de películas.
Por ejemplo, si analizamos el primer resultado ordenado por confianza:
{Client, The (1994), Natural Born Killers (1994)} -> {Fugitive, The (1993)} (conf: 0.931, supp: 0.052, lift: 2.535, conv: 9.198)
De aquí podemos deducir, que del total de usuarios que calificaron las primeras 2 películas, el 93% calificó además la tercera. El soporte de 0.052 nos indica que de total de usuarios analizados, solo el 0.00052% calificó las 3 películas.
Por otro lado, si analizamos ordenando por soporte:
{Indiana Jones and the Last Crusade (1989)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.813, supp: 0.188, lift: 2.013, conv: 3.185)
Este resultado nos muestra que casi el 20% de los usuarios han calificado ambas películas, lo cual es una cantidad muy significativa.
En cuanto al lift, si es igual a 1, podemos decir que las películas son independientes entre sí, en cambio cuando es mayor que 1, se puede decir que ambas películas son dependientes, en cambio si es menor que 1, significaría que son suplementarios. En el caso de las reglas obtenidas, esto no sucedió. De todas formas, significaría que los usuarios o califican X, o califican Y, y no nos interesarían estos valores.